In [3]:
import os
os.chdir('C:\\Users\\Takanori\\Desktop\\Kaggle\\TPS2205\\input')

# DataSet & Library Loading

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import time, gc

In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

In [6]:
class CFG:
    min_batch_size = 1_000  # max_batch_sizeの1000分の1程度が限界か
    max_batch_size = 1_000_000
    batch_increase_rate = 2
    epochs = 10_000
    folds = 10
    seed = 42
    target = 'target'
    model_path = "models"
    test_pred = ['pred' + str(i) for i in range(folds)]  # 各foldで作ったモデル別の、testを予測した結果のカラム名
    pred = 'pred'  # trainを予測した結果のカラム名
    increase_batch = 10  # 何epoch間lossが改善しなかったbatchを増やすか
    early_stopping = 200  # 何epochでearly stoppingをするか
    lr = 0.01
    lr_factor = 0.2  # lrを何倍にするか
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
os.makedirs(CFG.model_path, exist_ok=True)

## FE

In [8]:
# データセット読み込み
df_train = pd.read_csv("../input/tabular-playground-series-may-2022/train.csv")
df_test = pd.read_csv("../input/tabular-playground-series-may-2022/test.csv")
df_sub = pd.read_csv("../input/tabular-playground-series-may-2022/sample_submission.csv")
display(df_train.head())
display(df_test.head())
display(df_sub.head())

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,...,-2.540739,0.766952,-2.730628,-0.208177,1.363402,ABABDADBAB,67.609153,0,0,0
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,...,2.278315,-0.633658,-1.217077,-3.782194,-0.058316,ACACCADCEB,377.096415,0,0,1
2,2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,...,-1.385775,-0.520558,-0.009121,2.788536,-3.703488,AAAEABCKAD,-195.599702,0,2,1
3,3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,...,0.572594,-1.653213,1.686035,-2.533098,-0.608601,BDBBAACBCB,210.826205,0,0,1
4,4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,...,-3.912929,-1.430366,2.127649,-3.306784,4.371371,BDBCBBCHFE,-217.211798,0,1,1


,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30
0,900000,0.442517,0.174380,-0.999816,0.762741,0.186778,-1.074775,0.501888,6,6,...,-1.006400,-1.193879,-2.435736,-2.427430,-1.966887,5.734205,BAAABADLAC,99.478419,0,0
1,900001,-0.605598,-0.305715,0.627667,-0.578898,-1.750931,1.355550,-0.190911,1,3,...,2.382405,0.149442,1.883322,-2.848714,-0.725155,3.194219,AFABBAEGCB,-65.993825,1,0
2,900002,0.303990,2.445110,0.246515,0.818248,0.359731,-1.331845,1.358622,3,3,...,-7.026098,1.312277,-5.157192,1.714005,0.585032,0.066898,BBACABBKEE,-87.405622,0,1
3,900003,0.154053,0.260126,-1.367092,-0.093175,-1.111034,-0.948481,1.119220,0,0,...,-0.594532,-3.939475,1.754570,-2.364007,-1.003320,3.893099,AEBEAACQCC,-281.293460,0,0
4,900004,-1.651904,-0.424266,-0.667356,-0.322124,-0.089462,0.181705,1.784983,2,2,...,0.084906,-0.985736,-0.130467,-3.557893,1.210687,1.861884,AEBBBBDABF,25.629415,0,2


,id,target
0,900000,0.5
1,900001,0.5
2,900002,0.5
3,900003,0.5
4,900004,0.5


In [16]:
df_all = pd.concat([df_train, df_test])

In [10]:
# 不要データを削除
df_all.drop(['id'], axis=1, inplace=True)

In [11]:
# NA埋め
df_all.fillna(df_all.mean(numeric_only=True), inplace=True)

In [12]:
# f_27以外のカテゴリ変数をone_hot_encode
# 整数なので、一応値そのものは残しておく
cat_features = ['f_' + str(i).zfill(2) for i in range(7, 19)] + ['f_' + str(i).zfill(2) for i in range(29, 31)]
new_df = pd.get_dummies(df_all[cat_features].astype('category'))
df_all = pd.concat([df_all, new_df], axis=1)

In [1]:
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
lrm = LinearRegression()

In [ ]:
# 大体10分くらいかかる



tqdm.pandas()
df_all[['lr_a', 'lr_b']] = df_all['f_27'].progress_apply(calc_linear_regression)

In [17]:
# numpy配列にしてから回してみる
def c2i(c):
    return ord(c)-ord('A')

def calc_linear_regression(txt):
    arr = [c2i(c) for c in txt]
    y = np.array(arr).reshape(-1, 1)
    x = np.arange(len(y)).reshape(-1, 1)
    lrm.fit(x, y)
    return lrm.coef_[0].item(), lrm.intercept_.item()

tmp = df_all['f_27'].values
ret1 = []
ret2 = []
for i, txt in tqdm(enumerate(tmp)):
    a, b = calc_linear_regression(txt)
    ret1.append(a)
    ret2.append(b)


113301it [01:17, 1470.63it/s]


KeyboardInterrupt: 

In [11]:
# f_27を分解
def splitter(text):
    arr = tuple(text)
    return arr

column_list = ['f_27_' + str(i) for i in range(10)]
df_all[column_list] = df_all.apply(lambda x: splitter(x['f_27']), axis=1, result_type='expand')

In [12]:
# 新設したf_27を分解したものを、one-hot-encoding
new_df = pd.get_dummies(df_all[column_list])
df_all = pd.concat([df_all, new_df], axis=1)

In [13]:
# f_27_0~9を数値に
for column in column_list:
    df_all[column] = df_all[column].apply(lambda x: ord(x)-ord('A'))

In [14]:
# 線形回帰
# 遅すぎるので一旦飛ばす
def calc_linear_regression(pdseries):
    y = pdseries.values.astype(float)
    x = np.arange(len(y)).astype(float)
    result = np.polyfit(x, y, 1)
    return tuple(result.tolist())
# df_all[['f_27_linear_regression_a', 'f_27_linear_regression_b']] = df_all.apply(lambda x: calc_linear_regression(x[column_list]), axis=1, result_type='expand')


In [22]:
def calc_mean(pdseries):
    return pdseries.mean()

df_all['mean'] = df_all.apply(lambda x: calc_mean(x[column_list]), axis=1, result_type='expand')

In [15]:
# 不要になったカテゴリ変数を削除
df_all.drop(['f_27'], axis=1, inplace=True)

In [16]:
# この時点で数値しかない。正規化
scaler = StandardScaler()
for column in df_all.columns:
    if column == CFG.target:
        continue
    if df_all[column].dtype == 'object':
        continue
    df_all[column] = scaler.fit_transform(df_all[column].values.reshape(-1, 1))

In [17]:
df_train = df_all[:len(df_train)]
df_test = df_all[len(df_train):]

In [18]:
df_train.to_pickle('train_fe.pkl')
df_test.to_pickle('test_fe.pkl')

# Pytorch

In [19]:
df_train = pd.read_pickle('train_fe.pkl')
df_test = pd.read_pickle('test_fe.pkl')

## Define Pytorch Model

In [20]:
#thank you very much https://www.kaggle.com/mburakergenc/ttianic-minimal-pytorch-mlp
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        # 5層
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.bn3 = nn.BatchNorm1d(hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.bn4 = nn.BatchNorm1d(hidden_size)
        self.fc5 = nn.Linear(hidden_size, hidden_size)
        self.bn5 = nn.BatchNorm1d(hidden_size)
        self.fc6 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.dropout(F.silu(self.fc1(x)))
        x = self.bn1(x)
        x = self.dropout(F.silu(self.fc2(x)))
        x = self.bn2(x)
        x = self.dropout(F.silu(self.fc3(x)))
        x = self.bn3(x)
        x = self.dropout(F.silu(self.fc4(x)))
        x = self.bn4(x)
        x = self.dropout(F.silu(self.fc5(x)))
        x = self.bn5(x)
        x = self.fc6(x)
        return x

# Pytorch Loss Function

In [21]:
criterion = nn.BCEWithLogitsLoss()

# Pytorch Training

In [22]:
class LossChecker:
    # 指定した回数、lossが改善されていなければ打ち止めする
    def __init__(self, patience=20):
        self.patience = patience
        self.bef_epoch = 0
        self.min_loss = float('inf')

    def step(self, epoch, loss):
        if self.min_loss > loss:
            self.min_loss = loss
            self.bef_epoch = epoch
        if epoch - self.bef_epoch > self.patience:
            self.bef_epoch = epoch
            return True
        else:
            return False
early_stopping = LossChecker(CFG.early_stopping)
increase_batch = LossChecker(CFG.increase_batch)

In [24]:
# 学習の対象とする特徴量を列挙する
all_features = df_train.columns.tolist()
all_features.remove(CFG.target)

In [25]:
class CalcBatchSize:
    # バッチサイズを
    # 極大->最小->ちょっと増加->ちょっと増加->...の順で遷移させる
    def __init__(self, min_batch_size, max_batch_size, rate=1.2):
        self.min_batch_size = min_batch_size
        self.max_batch_size = max_batch_size
        self.first_batch_size = max_batch_size
        self.current_batch_size = -1
        self.cnt = -1
        self.rate = rate

    def step(self):
        self.cnt += 1
        if self.cnt == 0:
            self.current_batch_size = self.first_batch_size
            return self.current_batch_size
        elif self.cnt == 1:
            self.current_batch_size = self.min_batch_size
            return self.current_batch_size
        else:
            self.current_batch_size *= self.rate
            self.current_batch_size = int(self.current_batch_size) + 1
            self.current_batch_size = max(self.current_batch_size, self.min_batch_size)
            return self.current_batch_size


In [26]:
def fit(hidden_size, n_splits, df_train, df_test):
    # df_trainをシャッフル
    df_train = df_train.sample(frac=1).reset_index(drop=True)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=CFG.seed)

    for fold, (train_index, valid_index) in enumerate(skf.split(df_train, df_train[CFG.target])):
        print('-----', fold, '-----')
        # fold毎に初期化する設定
        # model, optimizer, scheduler, その他変数
        model = Net(len(all_features), hidden_size, 1).to(CFG.device)
        print(model)

        optimizer = torch.optim.Adam(model.parameters(), lr=CFG.lr)
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=CFG.lr_factor)
        calc_batch_size = CalcBatchSize(CFG.min_batch_size, CFG.max_batch_size, CFG.batch_increase_rate)
        batch_size = calc_batch_size.step()

        train_loss = 0
        train_loss_min = np.Inf

        # データを分割する。fold毎に一回だけやる
        X_train = df_train[all_features].iloc[train_index]
        y_train = df_train.iloc[train_index][CFG.target]
        X_valid = df_train[all_features].iloc[valid_index]
        y_valid = df_train.iloc[valid_index][CFG.target]
        X_test = df_test[all_features]
        st_time = time.time()

        for epoch in range(CFG.epochs):

            # TODO Variableを使うのは古いらしい
            x0_var = Variable(torch.FloatTensor(X_train.values)).to(CFG.device)
            y0_var = Variable(torch.FloatTensor(y_train.values)).to(CFG.device)
            i = -1
            while True:
                i += 1
                # ミニバッチ学習
                start = i * batch_size
                end   = start + batch_size
                x_var = x0_var[start:end]
                y_var = y0_var[start:end]
                if len(x_var) == 1:
                    print("len(x_var) == 1, cant use bn layer. skip this batch")
                    break
                if len(x_var) == 0:
                    # not need to train
                    break

                optimizer.zero_grad()
                output = model(x_var).squeeze(1)
                loss   = criterion(output, y_var)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()*batch_size
                if np.isnan(loss.item()):
                    print('-----nan-----')
                    print(loss.item(), torch.isnan(output).any(), torch.isnan(y_var).any())
                    raise "ERROR loss.item() is nan"

            train_loss = train_loss / len(X_train)
            if train_loss <= train_loss_min:
                print("Epoch: " + str(epoch+1).zfill(5), "batch_size: " + str(batch_size), "Validation loss decreased ({:6f} ===> {:6f}). Saving the model...".format(train_loss_min,train_loss))
                torch.save(model.state_dict(), CFG.model_path + "/model" + str(fold) + ".pt")
                train_loss_min = train_loss

            # batch_sizeを増やす ただし初回だけは学習速度向上の視点からbatch_sizeを非常に大きな値にしている
            # 暫くlossが変わってなければ、EarlyStoppingする。
            if increase_batch.step(epoch, train_loss):
                batch_size = calc_batch_size.step()
                print('change batch', batch_size)

            if early_stopping.step(epoch, train_loss):
                print('early stopping', epoch, train_loss)
                break

            # log
            if epoch % 200 == 0:
                print('')
                print("Epoch: " + str(epoch+1), "batch_size: " + str(batch_size), "Train Loss: {}".format(train_loss), "Time per epoch:", (time.time() - st_time)/(epoch+1))

        print('STOP train fold=', fold)
        x0_var = Variable(torch.FloatTensor(X_valid.values)).to(CFG.device)
        pred = torch.sigmoid(model(x0_var))
        df_train.loc[valid_index, CFG.pred] = pred.data.cpu().numpy()
        auc = roc_auc_score(df_train.loc[valid_index, CFG.target], df_train.loc[valid_index, CFG.pred])
        print('fold', fold, 'AUC:', auc)
        del model, optimizer, X_train, y_train, X_valid, y_valid, X_test, x0_var, y0_var, x_var, y_var, pred
        gc.collect()

    auc = roc_auc_score(df_train[CFG.target], df_train[CFG.pred])
    print('-----', 'Training Ended!', '-----')
    print('AUC', auc)

In [27]:
fit(32, 3, df_train, df_test)

----- 0 -----
Net(
  (fc1): Linear(in_features=346, out_features=32, bias=True)
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (bn4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=32, out_features=32, bias=True)
  (bn5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc6): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
Epoch: 00001 batch_size: 1000000 Validation loss decreased (   inf ===> 1.193654). Saving the model...

Epoch: 1 batch_size: 100000

KeyboardInterrupt: 

In [ ]:
fit(64, 10, df_train, df_test)

In [ ]:
fit(128, 10, df_train, df_test)

In [ ]:
fit(256, 10, df_train, df_test)

# predictions

In [ ]:
import glob

In [ ]:
models = np.sort(glob.glob(f"./{CFG.model_path}/*.pt"))
print(models)
# fold別に作った10個のモデルをロードする
with torch.no_grad():
    for i, model_name in enumerate(models):
        model = Net(len(all_features), 512, 1).to(CFG.device)
        model.load_state_dict(torch.load(model_name,))
        X_test = df_test[all_features]
        x0_var = Variable(torch.FloatTensor(X_test.values)).to(CFG.device)
        pred = F.sigmoid(model(x0_var))
        df_test[CFG.test_pred[i]] = pred.data.cpu().numpy()

df_test.head()


In [ ]:
df_sub[CFG.target] = df_test[CFG.test_pred].mean(axis=1)

In [ ]:
df_sub[CFG.target] = (df_sub[CFG.target] > 0.5).astype(int)

In [ ]:
df_sub.to_csv('submission.csv', index=False)

In [ ]:
df_train.to_csv('train_3layer_512.csv', index=False)
df_test.to_csv('test_3layer_512.csv', index=False)